In [9]:
# imports
import string
import torch
import pandas as pd
import datasets
from datasets import Dataset
# import torchtext

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\linda\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\linda\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [10]:
# load training dataset
train_df = pd.read_csv(r"../Dataset/zu-en.training.csv")
train_df.head(3)

,zu,en
0,"Lomkhakha kufanele uthuthukiswe, lawa amazwi k...","This sector needs to be developed, These are ..."
1,Yonke imibuzo: Ucingo: 031- 311 3154 (Shaks Ra...,All questions: Phone: 031- 311 3154 (Shaks Ram...
2,Axhumanisa umphakathi noMkhandlu ngoba abika k...,They connect the community with the Council be...


In [11]:
# load evaluation dataset
eval_df = pd.read_csv(r"../Dataset/zu-en.eval.csv")
eval_df.head(3)

,zu,en
0,Ikomidi elihlelela imidlalo ye-2013 Orange Afr...,The 2013 Orange Africa Cup of Nations (known a...
1,Futhi ipolitiki akuwona umdlalo wabantu abanga...,And politics is not a game for immature people.
2,Ikhasi lethu lakwa e-Careers likubeka ngokucac...,Our e-Careers page makes it clear that if you ...


In [12]:
# load test dataset
test_df = pd.read_csv(r"../Dataset/zu-en.test.csv")
test_df.head(3)

,zu,en
0,NONDUDUZO NGCONGO SEKUVELILE ukuthi ezokuvakas...,NONDUDUZO NGCONGO It has come to light that to...
1,Umkhankaso uzobe usezindaweni zokubhukuda ezis...,The campaign will be at the swimming pools in ...
2,SinguMasipala siyakuqonda ukukhala kwabantu ka...,As a Municipality we understand the cries of t...


In [13]:
# convert panda dataframes to HuggingFace datasets for easy data manipulation
train_data = Dataset.from_pandas(train_df)
eval_data = Dataset.from_pandas(eval_df)
test_data = Dataset.from_pandas(test_df)

# show features to work with
print(f"{train_data.features}\n{eval_data.features}\n{test_data.features}")

{'zu': Value(dtype='string', id=None), 'en': Value(dtype='string', id=None)}
{'zu': Value(dtype='string', id=None), 'en': Value(dtype='string', id=None)}
{'zu': Value(dtype='string', id=None), 'en': Value(dtype='string', id=None)}


In [14]:
# # tokenize function
# def tokenize(src_data, column):
#   data_copy = src_data.copy()
#   for index,row in data_copy.iterrows():
#     col_text = row[column]
#     col_text_without_punctuation = ''.join([char for char in col_text if char not in string.punctuation])
#     token = word_tokenize(col_text_without_punctuation)
#     data_copy.at[index, column] = token
#   return data_copy

# # tokenize zu
# df_copy = tokenize(df, 'zu')
# # zulu_tokenized = df_copy['zu']

# # tokenize en
# df_copy = tokenize(df_copy, 'en')
# # english_tokenized = df_copy['en']

# df_copy.head(3)

In [15]:
# function to tokenize a row (example) in dataset using .map function
def tokenize_example(example, max_length, lower, sos_token, eos_token):
    zu_tokens = word_tokenize(example["zu"])[:max_length]
    en_tokens = word_tokenize(example["en"])[:max_length]
    if lower:
        zu_tokens = [token.lower() for token in zu_tokens]
        en_tokens = [token.lower() for token in en_tokens]
    zu_tokens = [sos_token] + zu_tokens + [eos_token]
    en_tokens = [sos_token] + en_tokens + [eos_token]
    return {"zu_tokens": zu_tokens, "en_tokens": en_tokens}

    
# maybe try with spacy (treating zulu as english and tokenize both with en_nlp?)

In [16]:
max_length = 500
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "max_length": max_length,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs)
eval_data = eval_data.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(tokenize_example, fn_kwargs=fn_kwargs)

Map: 100%|██████████████████████████████████████████████████████████████████| 487/487 [00:00<00:00, 4638.16 examples/s]
